# Accelerating End-to-End Data Science Workflows # 

## 06 - XGBoost ##

**สารบัญ**
<br>
สมุดบันทึก (notebook) นี้จะใช้การบูสติ้งแบบไล่ระดับความชันที่เร่งความเร็วด้วย GPU เพื่อทำนายความน่าจะเป็นที่บุคคลหนึ่งติดเชื้อไวรัสจำลอง สมุดบันทึกนี้ครอบคลุมหัวข้อด้านล่าง:
1. [สภาพแวดล้อม(Environment)](#Environment)
2. [โหลดข้อมูล(Load-Data)](#Load-Data)
3. [แบ่งข้อมูลสำหรับฝึกและทดสอบ (Train-Test-Split)](#Train-Test-Split)
4. [XGBoost](#XGBoost)
    * [การตั้งค่าพารามิเตอร์ XGBoost](#Setting-XGBoost-Parameters)
    * [การฝึกโมเดล](#Training-the-Model)
5. [การตรวจสอบโมเดล](#Inspecting-the-Model)
6. [การทำนายผล](#Making-Predictions)
7. [(ทางเลือก) การเปรียบเทียบ: XGBoost เฉพาะ CPU](#(Optional)-Comparison:-CPU-Only-XGBoost)

## สภาพเเวดล้อม ##
เราจะใช้ไลบรารี [XGBoost](https://xgboost.readthedocs.io/en/latest/) เพื่อสร้างโมเดล Gradient Boosted สำหรับแบบฝึกหัดนี้

นอกเหนือจากส่วนประกอบปกติของ RAPIDS แล้ว เรายังอิมพอร์ตไลบรารีหลายตัวที่จะช่วยให้เราเข้าใจและประเมินโมเดล XGBoost หลังจากที่เราได้ฝึกฝนมันแล้ว

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cudf
import cuml
import cupy as cp

from cuml.model_selection import train_test_split

# visualization
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import graphviz

# model analysis
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import xgboost as xgb

# xgboost version 1.0 or later is required to directly convert from cudf Dataframes to xgboost DMatrix format
print('XGBoost version: ', xgb.__version__)

## โหลดข้อมูล ##
สำหรับหน้านี้ (notebook) เราจะโหลดข้อมูลประชากรมาเพียงบางส่วน ซึ่งรวมถึงทั้งคอลัมน์ที่เราเคยใช้สำหรับการถดถอยโลจิสติกส์ (logistic regression) และคอลัมน์พิกัดด้วย XGBoost ช่วยให้เราสามารถใช้ข้อมูลที่มีความสัมพันธ์แบบไม่เชิงเส้น (nonlinear relationships) กับผลลัพธ์ที่เราสนใจได้ และข้อมูลเชิงพื้นที่ (geospatial data) มักจะจัดอยู่ในประเภทนั้น

In [ ]:
gdf = cudf.read_csv('./data/clean_uk_pop_full.csv', usecols=['age', 'sex', 'northing', 'easting', 'infected'])

In [ ]:
gdf.dtypes

In [ ]:
gdf.shape

In [ ]:
gdf.head()

ก่อนที่เราจะแบ่งข้อมูลสำหรับฝึกฝน (training) และทดสอบ (testing) เราจะตรวจสอบสถานะหน่วยความจำของเราครับ เราต้องการให้การใช้งานหน่วยความจำอยู่ต่ำกว่าครึ่งหนึ่งของหน่วยความจำทั้งหมดบน GPU ที่กำลังใช้งาน เพื่อให้การเพิ่มขึ้นชั่วคราวจากการแบ่งข้อมูลยังคงสามารถจัดเก็บในหน่วยความจำได้

In [ ]:
!nvidia-smi

## การแบ่งข้อมูล (Train-Test Split) ##
เราจะใช้วิธีการแบ่งข้อมูล (splitting method) อีกครั้งเพื่อสร้างชุดข้อมูลย่อยสำหรับ **การฝึก (training)** และ **การทดสอบ (testing)** โดยคำนึงว่าการทำเช่นนี้จะใช้หน่วยความจำเพิ่มขึ้น

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(gdf[['age', 'sex', 'northing', 'easting']], gdf['infected'])

In [ ]:
!nvidia-smi

**ข้อสังเกต**: ตอนนี้เรามีชุดข้อมูลสำหรับ **ฝึก (training)** และ **ทดสอบ (testing)** แล้ว เราสามารถลบข้อมูลต้นฉบับออกได้ เพื่อเพิ่มพื้นที่ว่างสำหรับหน่วยความจำเสริมของอัลกอริทึม ซึ่งในกรณีนี้อาจจะไม่สำคัญนัก แต่ก็เป็นวิธีปฏิบัติที่เป็นประโยชน์เมื่อพยายามฝึกโมเดลด้วยข้อมูลให้ได้มากที่สุดเท่าที่จะทำได้

In [ ]:
del(gdf)

## XGBoost ##

### การตั้งค่าพารามิเตอร์ XGBoost ###
ตอนนี้เราสามารถตั้งค่าพารามิเตอร์สำหรับการรันเทรน XGBoost นี้ได้แล้ว โดยพารามิเตอร์เหล่านี้จะกำหนดประเภทและขนาดของต้นไม้ที่จะถูกสร้างขึ้น รวมถึงวิธีการที่เราใช้วัดผลความสำเร็จ

พารามิเตอร์ที่สำคัญคือ `cuda`: ซึ่งเป็นการบอก XGBoost ว่าเราต้องการให้การเทรนทำงานบน **GPU**

สำหรับกรณีการใช้งานของเรา เราต้องการทำนายความน่าจะเป็นที่บุคคลจะติดเชื้อไวรัสอีกครั้ง ดังนั้นเราจึงตั้งค่า **objective** เป็น `binary:logistic` (ผลลัพธ์แบบไบนารี โดยใช้วิธีโลจิสติกเพื่อหาความน่าจะเป็น)

ตัวเลือกพารามิเตอร์และความหมายอื่นๆ สามารถดูได้ที่ [พารามิเตอร์ XGBoost](https://xgboost.readthedocs.io/en/latest/parameter.html)

In [ ]:
params = {
    'max_depth':    8,
    'max_leaves':   2**8,
    'device': 'cuda',
    'tree_method':  'hist',
    'objective':    'binary:logistic',
    'grow_policy':  'lossguide',
    'eval_metric':  'logloss',
    'subsample':    '0.8'
}

### การฝึกโมเดล (Training the Model) ###
XGBoost ใช้โครงสร้างข้อมูลพิเศษที่มีประสิทธิภาพสูงที่เรียกว่า `DMatrix` ดังนั้นเราจึงส่ง DataFrame สำหรับการฝึก (training dataframes) เข้าไปเพื่อสร้าง `DMatrix`

โปรดทราบว่าข้อมูลยังคงอยู่บน GPU โดยจะส่งผ่านจาก cuDF ไปยัง XGBoost โดยตรง

In [ ]:
dtrain = xgb.DMatrix(x_train, y_train)

ทีนี้ เราก็พร้อมที่จะ **ฝึกโมเดล (train the model)** แล้ว

In [ ]:
%time model = xgb.train(params, dtrain, num_boost_round=100)

In [ ]:
model.save_model('xgboost_model.json')

In [ ]:
print(x_train)
print(y_train)

**ข้อสังเกต**: เพื่อใช้เป็นจุดเปรียบเทียบ โค้ดสำหรับรัน XGBoost เวอร์ชันที่ใช้ CPU เท่านั้น จะมีให้ที่ส่วนท้ายของแบบฝึกหัดนี้

## ตรวจสอบโมเดล ##
เราสามารถตรวจสอบโมเดลได้หลายวิธี ก่อนอื่น เราสามารถดูว่าฟีเจอร์ใดที่โมเดลเชื่อว่าสำคัญที่สุดในการประเมิน คะแนน F ที่สูงขึ้นบ่งชี้ถึงความสำคัญที่ประเมินไว้สูงขึ้น

ดูเหมือนว่ามีองค์ประกอบทาง **ภูมิสารสนเทศ (geospatial)** ที่แข็งแกร่งต่อการกระจายของการติดเชื้อ เนื่องจากฟีเจอร์ **easting** และ **northing** มีคะแนน F สูงที่สุด นอกจากนี้ **อายุ (age)** ดูเหมือนจะมีผลกระทบมากกว่า **เพศ (sex)** ในการกำหนดอัตราการติดเชื้อ (ซึ่งสอดคล้องกับผลลัพธ์ที่เราได้รับจากการวิเคราะห์ Logistic Regression)

In [ ]:
ax = xgb.plot_importance(model, height=.8)
ax.grid(False)
ax.set_title('F score by feature')
plt.show()

เรายังสามารถดึง **แต่ละต้นไม้ (individual trees)** ออกมาจากโมเดล และดูว่ามันใช้การตัดสินใจอะไรบ้างในการสนับสนุนผลลัพธ์โดยรวม (ensemble) โปรดสังเกตว่า เช่นเดียวกับวิธีการแบบ **ensemble** อื่นๆ ต้นไม้แต่ละต้นอาจดูเหมือนไม่ได้สร้างความแตกต่างอย่างมีนัยสำคัญในผลลัพธ์ (ค่าบนโหนดใบไม้) แต่การรวมกันของต้นไม้ที่แต่ละต้นอาจอ่อนแอหลายๆ ต้นเข้าด้วยกันให้เป็นโมเดลที่แข็งแกร่ง คือสิ่งที่ทำให้ XGBoost มีประสิทธิภาพ

ลองเปลี่ยนค่า `num_trees` เพื่อตรวจสอบต้นไม้ที่แตกต่างกันในโมเดล การเปลี่ยน `rankdir` เป็น `'TB'` จะปรับทิศทางของต้นไม้ให้เป็นแบบบนลงล่าง

In [ ]:
xgb.plot_tree(model, num_trees=0, rankdir='LR')

# get current figure to set the size
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(100, 100)

## การคาดการณ์ (Making Predictions) ##
เมื่อเราคุ้นเคยกับโมเดลแล้ว เราก็จะเริ่มทำการคาดการณ์ด้วยโมเดลนั้น เรายืนยันว่าจะทำการคาดการณ์บนข้อมูลมากกว่า 11 ล้านแถว

In [ ]:
x_test.shape

เราจะแปลงเมทริกซ์ *X* ให้เป็น `DMatrix` เหมือนเดิม จากนั้นทำการคาดการณ์ (prediction) สำหรับแต่ละแถว **สังเกตเวลาที่ใช้ในการคาดการณ์กว่า 11 ล้านรายการ**

In [ ]:
dtest = xgb.DMatrix(x_test)
%time y_pred = model.predict(dtest)

ตอนนี้เราต้องการดูว่าการทำนายเหล่านั้นดีแค่ไหน วิธีการประเมินทั่วไปคือการคำนวณ **พื้นที่ใต้กราฟ (AUC)** ของ **กราฟคุณลักษณะการทำงานของตัวรับ (ROC curve)**

การทำนายเป็นอาร์เรย์ `numpy` ดังนั้นเราจึงแปลงป้ายกำกับการทดสอบให้ตรงกัน จากนั้นจึงทำการคำนวณ ROC curve

In [ ]:
y_test_cpu = cp.asnumpy(cp.array(y_test))
false_pos_rate, true_pos_rate, thresholds = roc_curve(y_test_cpu, y_pred)

สุดท้าย เราสามารถพล็อต เส้นโค้ง (curve) และคำนวณ คะแนน AUC (AUC score) เพื่อช่วยให้เราประเมินความสัมพันธ์ระหว่างอัตราการเกิดผลบวกจริง (true positive rates) และอัตราการเกิดผลบวกปลอม (false positive rates) ได้

In [ ]:
auc_result = auc(false_pos_rate, true_pos_rate)

fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(false_pos_rate, true_pos_rate, lw=3,
        label='AUC = {:.2f}'.format(auc_result))
ax.plot([0, 1], [0, 1], 'k--', lw=2)
ax.set(
    xlim=(0, 1),
    ylim=(0, 1),
    title="ROC Curve",
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
)
ax.legend(loc='lower right');
plt.show()

## (ทางเลือก) การเปรียบเทียบ: XGBoost เฉพาะ CPU ##
ด้านล่างนี้ เราได้เตรียมโค้ดสำหรับการฝึก (training) และการอนุมาน (inferring) โดยใช้ XGBoost เฉพาะ CPU ซึ่งใช้พารามิเตอร์โมเดลเดียวกัน เพียงแต่เปลี่ยนวิธีการสร้างฮิสโตแกรมของทรี (histogram tree method) จาก GPU เป็น CPU เท่านั้น

In [ ]:
params['device'] = 'cpu'
dtrain_cpu = xgb.DMatrix(x_train.to_pandas(), y_train.to_pandas())
%time model_cpu = xgb.train(params, dtrain_cpu, num_boost_round=100)

In [ ]:
dtest_cpu = xgb.DMatrix(x_test.to_pandas())
%time y_pred_cpu = model_cpu.predict(dtest_cpu)

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**เยี่ยมมาก!** ไปยัง [สมุดบันทึกถัดไป](3-07_triton.ipynb) กันเลย